In [1]:
# "기존의 항암 치료가 평균적인 환자를 위한 접근 방식이었다면,
# 본 프로젝트는 환자의 유전자 데이터를 분석하여 암세포의 고유한 특성을 파악하고
# 개인 맞춤형 치료 전략을 설계하는 데 기여하는 것을 목표로 합니다."

# AI를 통한 폐암 암세포 신항원 발굴 및 암살자(면역세포) 설계.
# 딥러닝 모델의 Output(결과값)이 왜 "이 단백질이 신항원일 확률

# "신항원(Neoantigen) /"암세포가 가진 고유한 위장 실패의 흔적"
# 암세포가 끊임없이 돌연변이를 일으키는 과정에서 만들어진 완전히 새로운(Neo) 단백질 조각(Antigen)"발굴 

# 암살자: 암세포의 특정 단백질(신항원)을 식별하여 정밀하게 공격하고 파괴하는 면역 세포(T세포)
# 암세포는 계속 변해서 면역세포가 못 알아챔. 
# AI의 역할: 환자의 유전자 서열(텍스트 데이터)을 분석해서,
# 면역세포가 '적'으로 인식할 수 있는 최적의 단백질 조각(신항원)을 AI가 디자인
# 암백신 설계도 역할.

# 1D-CNN

In [2]:
# 1. 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import os

c:\code\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [3]:
# 폰트지정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 마이너스 부호 깨짐 지정
plt.rcParams['axes.unicode_minus'] = False


# 파일이 너무 커서 csv 파일 읽어오는데 너무 시간이 걸림 - > 파일 변환해서 시간 단축하기 위해 이부분 추가 진행
# 1. 처음 한 번만 실행: CSV를 불러와서 필요한 것만 챙긴 뒤 '파켓'으로 저장
# df = pd.read_csv('dataset/mhc_ligand_full.csv', low_memory=False)

# Parquet 형식으로 저장 (압축률이 엄청납니다)
# 만약 에러가 나면 !pip install pyarrow .
# df.to_parquet('dataset/mhc_data.parquet')
# print("축하합니다! 이제 mhc_data.parquet 파일이 생성되었습니다.")

# 데이터가 너무 커서 컴에서 읽지 못하는 경우 아래 방법으로 진행
# csv_file = 'dataset/mhc_ligand_full.csv'
# parquet_file = 'dataset/mhc_data.parquet'
# chunk_size = 100000  # 한 번에 10만 행씩 읽기 (메모리 상황에 따라 조절)

# # 1. Parquet 저장을 위한 첫 루프
# first_chunk = True
# for chunk in pd.read_csv(csv_file, low_memory=False, chunksize=chunk_size):
#     if first_chunk:
#         # 첫 번째 청크는 파일을 새로 생성
#         chunk.to_parquet(parquet_file, engine='pyarrow', index=False)
#         first_chunk = False
#     else:
#         # 이후 청크는 기존 파일에 추가 (단, fastparquet 엔진 권장 혹은 append 방식 고려)
#         # 일반적인 to_parquet는 덮어쓰기가 기본이므로 아래의 '방법 2'를 더 추천합니다.
#         pass

# print("변환 완료!")

# 2. 데이터 로드
df = pd.read_parquet('dataset/mhc_data.parquet')

In [4]:
# 실제 컬럼명 출력
print("="*50)
print("📂 전체 컬럼명 목록")
print("-" * 50)
for i, col in enumerate(df.columns):
    print(f"{i+1}. {col}")
print("="*50)

df = pd.read_parquet('dataset/mhc_data.parquet')

📂 전체 컬럼명 목록
--------------------------------------------------
1. Assay ID
2. Reference
3. Reference.1
4. Reference.2
5. Reference.3
6. Reference.4
7. Reference.5
8. Reference.6
9. Reference.7
10. Epitope
11. Epitope.1
12. Epitope.2
13. Epitope.3
14. Epitope.4
15. Epitope.5
16. Epitope.6
17. Epitope.7
18. Epitope.8
19. Epitope.9
20. Epitope.10
21. Epitope.11
22. Epitope.12
23. Epitope.13
24. Epitope.14
25. Epitope.15
26. Epitope.16
27. Epitope.17
28. Epitope.18
29. Related Object
30. Related Object.1
31. Related Object.2
32. Related Object.3
33. Related Object.4
34. Related Object.5
35. Related Object.6
36. Related Object.7
37. Related Object.8
38. Related Object.9
39. Related Object.10
40. Related Object.11
41. Related Object.12
42. Related Object.13
43. Related Object.14
44. Host
45. Host.1
46. Host.2
47. Host.3
48. Host.4
49. Host.5
50. Host.6
51. in vivo Process
52. in vivo Process.1
53. in vivo Process.2
54. in vivo Process.3
55. in vivo Antigen
56. in vivo Antigen.1
57. in vivo A

In [5]:
# 컬럼찾기(컬럼명이 이상해서 대표 5개 데이터로 추가 확인)
print(df.head())

                        Assay ID                          Reference  \
0                       IEDB IRI                           IEDB IRI   
1   http://www.iedb.org/assay/26  http://www.iedb.org/reference/274   
2  http://www.iedb.org/assay/115  http://www.iedb.org/reference/299   
3  http://www.iedb.org/assay/143  http://www.iedb.org/reference/304   
4  http://www.iedb.org/assay/144  http://www.iedb.org/reference/304   

  Reference.1 Reference.2    Reference.3  \
0        Type        PMID  Submission ID   
1  Literature    15448372           None   
2  Literature    15140958           None   
3  Literature    15102821           None   
4  Literature    15102821           None   

                                         Reference.4   Reference.5  \
0                                            Authors       Journal   
1  Yi-Hsiang Huang; Mi-Hua Tao; Cheng-Po Hu; Wan-...   J Gen Virol   
2  Yue-Dan Wang; Wan-Yee Fion Sin; Guo-Bing Xu; H...       J Virol   
3  Alberto Diaz-Quiñonez; Na

In [6]:

# ---------------------------------------------------------
# ✅ [최종 수정] KeyError 방지용 위치 기반 필터링
# ---------------------------------------------------------
# 컬럼 이름 대신 '위치(8번째)'를 사용하여 키워드 검사
# .iloc[:, 8]은 모든 행의 8번째 컬럼 데이터를 의미합니다.
is_lung_cancer = df.iloc[:, 8].str.contains('Lung|Adenocarcinoma|NSCLC|Cancer', case=False, na=False)

# 이제 인덱스 오류 없이 폐암 데이터만 골라냅니다.
df_lung = df[is_lung_cancer].copy()


# 딱 5줄만 가져와서 모든 칸에 뭐가 들어있는지 번호별로 봅니다.
# 여기서 'Positive'나 'Negative'라는 단어가 있는 번호를 찾아야 합니다. 데이터가 너무 많아서 결과를 찾는 함수
test_row = df_lung.iloc[0] 
for i, val in enumerate(test_row):
    if 'Positive' in str(val) or 'Negative' in str(val):
        print(f"🎯 결과(Label) 번호: [{i}], 내용: {val}")

# ---------------------------------------------------------
# [단계 1] 확보된 폐암 데이터에서 필요한 칸(서열, 결과) 추출
# ---------------------------------------------------------# [수정된 Cell 6 로직]
# 1. 폐암 데이터 중 Positive와 Negative를 각각 추출
df_pos = df_lung[df_lung.iloc[:, 94].str.contains('Positive', case=False, na=False)].copy()
df_neg = df_lung[df_lung.iloc[:, 94].str.contains('Negative', case=False, na=False)].copy()

# 2. 데이터 불균형 해결 (Negative 데이터가 적다면 전체 데이터에서 정상 샘플을 더 가져와야 함)
# 여기서는 학습의 질을 위해 1:1 비율로 맞춤 샘플링을 진행합니다.
sample_size = min(len(df_pos), len(df_neg))
df_pos_sampled = df_pos.sample(n=sample_size, random_state=42)
df_neg_sampled = df_neg.sample(n=sample_size, random_state=42)

# 3. 두 데이터를 합쳐서 최종 학습셋 구성
df_final = pd.concat([df_pos_sampled, df_neg_sampled])
df_final = df_final.iloc[:, [11, 94]] # 서열과 라벨만 추출
df_final.columns = ['Sequence', 'Label']

# 9글자 필터링
df_final = df_final[df_final['Sequence'].str.len() == 9].dropna()
print(f"✅ 균형 잡힌 데이터 확보: Positive {sample_size}건 / Negative {sample_size}건")

# 기존 df_final을 복사해서 분석용 변수들을 추가합니다.
df_report = df_final.copy()

# 1. 서열의 길이 (모두 9겠지만 통계에는 잡힙니다)
df_report['Seq_Length'] = df_report['Sequence'].str.len()

# 2. 특정 아미노산의 포함 여부 (예: 암세포와 관련 깊은 'L', 'V' 등)
df_report['Has_Leucine(L)'] = df_report['Sequence'].str.contains('L').astype(int)
df_report['Has_Valine(V)'] = df_report['Sequence'].str.contains('V').astype(int)

# 3. 서열의 첫 번째와 마지막 아미노산 (위치별 특성 파악용)
df_report['First_AA'] = df_report['Sequence'].str[0]
df_report['Last_AA'] = df_report['Sequence'].str[-1]

# 4. 아미노산의 소수성(Hydrophobicity) 대략적 분류 (간단 예시)
# 서열에 특정 문자가 많으면 1, 아니면 0 식으로 특징 추출
df_report['Hydrophobic_Score'] = df_report['Sequence'].apply(lambda x: sum(1 for aa in x if aa in 'AILMFPWV'))

# 이제 변수가 늘어난 데이터로 리포트 생성!
from ydata_profiling import ProfileReport
profile = ProfileReport(df_report, title="폐암 신항원 정밀 분석 리포트")
profile.to_file("lung_cancer_rich_report.html")

print("-" * 50)
print(f"✅ 최종 9-mer 폐암 데이터 확보: {len(df_final)}건")
print("-" * 50)
print(df_final.head())

🎯 결과(Label) 번호: [94], 내용: Positive
✅ 균형 잡힌 데이터 확보: Positive 6건 / Negative 6건


c:\code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 333.94it/s]

--------------------------------------------------
✅ 최종 9-mer 폐암 데이터 확보: 10건
--------------------------------------------------
       Sequence                  Label
6539  GLACHQLCA  Positive-Intermediate
5140  YMNGTMSQV          Positive-High
5132  LLMGTLGIV          Positive-High
6536  FLWGPRALV  Positive-Intermediate
6617  YLSGANLNL           Positive-Low


In [7]:
# 1. 아미노산 빈도 히트맵
def draw_heatmap(df):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    heatmap_data = np.zeros((20, 9))
    for seq in df['Sequence']:
        for i, aa in enumerate(seq):
            if aa in amino_acids:
                heatmap_data[amino_acids.index(aa), i] += 1
    plt.figure(figsize=(10, 7))
    sns.heatmap(heatmap_data, xticklabels=range(1, 10), yticklabels=list(amino_acids), cmap='YlGnBu')
    plt.title('신항원 서열 위치별 아미노산 분포 (Heatmap)')
    plt.show()

# 2. 정답 레이블 분포 (히스토그램/바 차트)
def draw_label_dist(df):
    plt.figure(figsize=(6, 4))
    sns.countplot(x='Label', data=df)
    plt.title('면역 반응 정답(Label) 분포')
    plt.show()

# 실행
draw_heatmap(df_final)
draw_label_dist(df_final)

C:\Users\human-24\AppData\Local\Temp\ipykernel_15540\1817969079.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\human-24\AppData\Local\Temp\ipykernel_15540\1817969079.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# 1. 아미노산 인코딩 함수 (노트북 분석 결과 반영)
def encode_sequence(seq):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY' # 20개 표준 순서
    aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
    matrix = np.zeros((9, 20))
    for i, aa in enumerate(seq.upper()):
        if aa in aa_to_int:
            matrix[i, aa_to_int[aa]] = 1
    return matrix

# 1-2. 리스트 전체 변환 함수
def encode_all_sequences(sequences):
    encoded_list = [encode_sequence(s) for s in sequences]
    return np.array(encoded_list)

# 2. 모델 생성 함수 정의 (개선된 1D-CNN 구조)
def build_improved_model():
    model = models.Sequential([
        # 첫 번째 층: 서열의 지역적 특징 포착
        layers.Conv1D(64, kernel_size=3, padding='same', activation='relu', input_shape=(9, 20)),
        layers.BatchNormalization(), # 학습 안정화
        
        # 두 번째 층: 더 넓은 패턴 포착
        layers.Conv1D(128, kernel_size=2, padding='same', activation='relu'),
        layers.GlobalMaxPooling1D(), # 위치 무관 특징 추출
        
        # 의사결정 층
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3), # 과적합 방지
        layers.Dense(1, activation='sigmoid') # 확률 출력
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 3. 데이터 준비 및 모델 생성 실행
# X, y 데이터 생성
X = encode_all_sequences(df_final['Sequence'].values)
y = (df_final['Label'].str.contains('Positive', case=False)).astype(int).values

# 데이터셋 분리 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성 (이제 함수가 위에서 정의되었으므로 에러가 나지 않습니다)
model = build_improved_model()

print(f"✅ 학습 데이터 준비 완료: {X_train.shape}")
print(f"✅ 신항원(Positive) 샘플 비중: {y.mean()*100:.2f}%")

✅ 학습 데이터 준비 완료: (8, 9, 20)
✅ 신항원(Positive) 샘플 비중: 50.00%


c:\code\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# ---------------------------------------------------------
# ✅ [9번 셀 수정] 개선된 모델로 학습 진행
# ---------------------------------------------------------

# 1. 9번 셀 상단의 모델 재정의(Sequential...) 부분을 삭제합니다.
# 2. 대신 8번 셀에서 이미 생성된 'model' 객체를 그대로 사용합니다.

# 모델 설정 (학습 지표 추가)
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# 3. 모델 학습 시작 
print("🚀 개선된 AI 모델로 학습을 시작합니다...")

# 실제 신항원을 더 잘 맞히기 위해 가중치(class_weight)를 추가하는 것이 좋습니다.
# Positive(1) 데이터를 더 엄격하게 배우도록 2.5배 가중치를 둡니다.
history = model.fit(X_train, y_train, 
                    epochs=20,           # 학습 횟수를 조금 더 늘려 숙련도를 높입니다.
                    batch_size=32, 
                    validation_data=(X_test, y_test),
                    class_weight={0: 1.0, 1: 2.5}) # 실제 항원을 놓치지 않게 강조

# 4. 결과 시각화 코드는 동일하게 유지

🚀 개선된 AI 모델로 학습을 시작합니다...
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 1.4774 - precision: 0.5000 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.7117 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5000 - loss: 1.1762 - precision: 0.5000 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.7068 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5000 - loss: 0.9041 - precision: 0.5000 - recall: 0.7500 - val_accuracy: 0.5000 - val_loss: 0.7038 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7500 - loss: 0.6736 - precision: 0.6667 - recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.7030 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7500 - loss: 0.7991 - precision: 0.6667 - recall: 1.0000 - val_accuracy: 0.5000 -

In [10]:
# 1. 모델 평가 (지표가 여러 개이므로 리스트로 받습니다)
evaluation_results = model.evaluate(X_test, y_test)

# 모델 컴파일 시 설정한 순서대로 값이 들어옵니다: [Loss, Accuracy, Precision, Recall]
loss = evaluation_results[0]
accuracy = evaluation_results[1]
precision = evaluation_results[2]
recall = evaluation_results[3]

print(f"📊 최종 테스트 손실(Loss): {loss:.4f}")
print(f"🎯 최종 테스트 정확도(Accuracy): {accuracy*100:.2f}%")
print(f"⚖️ 정밀도(Precision): {precision*100:.2f}%  <- '암'이라고 한 것 중 진짜 암인 비율")
print(f"📈 재현율(Recall): {recall*100:.2f}%  <- 실제 암 서열을 놓치지 않고 찾아낸 비율")

# 2. 샘플 데이터 예측 수행 (이 부분은 동일합니다)
samples = X_test[:5]
predictions = model.predict(samples)

print("\n--- 🔬 신항원 예측 수행 결과 (샘플) ---")

# predictions에 들어있는 실제 개수만큼만 반복합니다.
# 만약 5개보다 많을 때 딱 5개만 보고 싶다면 min(len(predictions), 5)를 쓰세요.
num_samples = len(predictions) 

for i in range(num_samples):
    # predictions[i]가 배열 형태인 경우와 단일 값인 경우를 모두 고려
    prob = predictions[i][0] if hasattr(predictions[i], "__len__") else predictions[i]
    
    result = "Positive" if prob > 0.5 else "Negative"
    probability = prob * 100
    print(f"서열 샘플 {i+1}번 -> 예측 결과: {result} (확률: {probability:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5000 - loss: 0.7161 - precision: 0.5000 - recall: 1.0000
📊 최종 테스트 손실(Loss): 0.7161
🎯 최종 테스트 정확도(Accuracy): 50.00%
⚖️ 정밀도(Precision): 50.00%  <- '암'이라고 한 것 중 진짜 암인 비율
📈 재현율(Recall): 100.00%  <- 실제 암 서열을 놓치지 않고 찾아낸 비율
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step

--- 🔬 신항원 예측 수행 결과 (샘플) ---
서열 샘플 1번 -> 예측 결과: Positive (확률: 59.52%)
서열 샘플 2번 -> 예측 결과: Positive (확률: 58.99%)


In [11]:
from ydata_profiling import ProfileReport

# 1. 리포트 생성 (반드시 df_report를 넣어야 8개 변수가 보입니다!)
profile = ProfileReport(df_report, 
                        title="폐암 신항원 정밀 분석 리포트 (8개 변수)", 
                        explorative=True,
                        minimal=False)

# 2. HTML 파일로 저장
profile.to_file("lung_cancer_FINAL_report.html")

# 3. 완료 메시지 (파일 이름 일치시키기)
print("✨ 데이터 분석 리포트가 'lung_cancer_FINAL_report.html'로 저장되었습니다!")

Export report to file: 100%|██████████| 1/1 [00:00<?, ?it/s]

✨ 데이터 분석 리포트가 'lung_cancer_FINAL_report.html'로 저장되었습니다!


In [12]:
# 기존 .h5 대신 최신 표준인 .keras를 사용합니다.
model.save("lung_cancer_model.keras")

print("✨ 모델이 'lung_cancer_model.keras'로 안전하게 저장되었습니다!")

✨ 모델이 'lung_cancer_model.keras'로 안전하게 저장되었습니다!
